In [ ]:
%%time
#SPINNEYS DATA SCRAPPING
import re
import urllib.request,urllib.parse,urllib.error
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
metascore_w = re.compile('metascore_w.*')
Rating= re.compile('cert_rating.*')

url='https://www.metacritic.com/browse/movies/score/metascore/all'
base='https://www.metacritic.com'
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,}
Movies = pd.DataFrame(columns=['Number','Title','Ratingstatus','Realeasedate','metascore','userscore','movielink'])
request=urllib.request.Request(url,None,headers)
response=urllib.request.urlopen(request)
soup=BeautifulSoup(response,'html.parser')
i=0
while (request):
  i=i+1
  try:
    Hirerchicalcont=soup.body.findAll('td',{"clamp-summary-wrap"})
    for item in Hirerchicalcont:
      Number=item.findAll('span',{"title numbered"})
      Titles=item.findAll('a',{"title"})
      ratingstatus=item.findAll('span',{"class":Rating})
      realeasedate=item.findAll('div',{"clamp-details"})
      metascores=item.findAll('div',{"clamp-metascore"})
      userscores=item.findAll('div',{"clamp-userscore"})
      if ratingstatus is not None and len(ratingstatus) > 0:
        rating = ratingstatus[0].text
      else:
        rating='null'
      Movies = Movies.append({"Number":Number[0].text.strip()[:-1],"Title":Titles[0].text,"Ratingstatus": rating[2:], "Realeasedate":realeasedate[0].span.text,"metascore":
               metascores[0].find('div',{"class":metascore_w}).text,"userscore":userscores[0].find('div',{"class":metascore_w}).text,"movielink":base+Titles[0]['href']},ignore_index=True)
  except IndexError:
    continue
  try:
      request=urllib.request.Request(url+'?page='+ str(i),None,headers)
      print(url+'?page='+ str(i))
      response=urllib.request.urlopen(request)
      soup=BeautifulSoup(response,'html.parser')
      time.sleep(3)
  except IndexError:
    break



In [47]:
Movies.to_csv('Movies.csv',index=False)